In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Параметры
a = 3
sigma = 1
gamma = 0.99
n = 10
M = 2000
K = 120

# Шаг 1: Смоделировать выборку из n независимых наблюдений над случайной величиной X
X = np.random.normal(a, sigma, n)

In [ ]:
z_value = stats.norm.ppf(1 - (1 - gamma) / 2)
mean_X = np.mean(X)
conf_interval_known_sigma = (mean_X - z_value * sigma / np.sqrt(n), mean_X + z_value * sigma / np.sqrt(n))

print("Интервальная оценка для математического ожидания X (известная дисперсия, формулы):", conf_interval_known_sigma)

In [ ]:
conf_interval_known_sigma_scipy = stats.norm.interval(alpha=gamma, loc=mean_X, scale=sigma/np.sqrt(n))
print("Интервальная оценка для математического ожидания X (известная дисперсия, scipy):", conf_interval_known_sigma_scipy)

1.2. Интервальная оценка для математического ожидания X при неизвестной дисперсии

In [ ]:
t_value = stats.t.ppf(1 - (1 - gamma) / 2, df=n-1)
std_X = np.std(X, ddof=1)
conf_interval_unknown_sigma = (mean_X - t_value * std_X / np.sqrt(n), mean_X + t_value * std_X / np.sqrt(n))

print("Интервальная оценка для математического ожидания X (неизвестная дисперсия, формулы):", conf_interval_unknown_sigma)

In [ ]:
conf_interval_unknown_sigma_scipy = stats.t.interval(alpha=gamma, df=n-1, loc=mean_X, scale=std_X/np.sqrt(n))
print("Интервальная оценка для математического ожидания X (неизвестная дисперсия, scipy):", conf_interval_unknown_sigma_scipy)

1.3. Интервальная оценка для дисперсии X

In [ ]:
chi2_lower = stats.chi2.ppf((1 - gamma) / 2, df=n-1)
chi2_upper = stats.chi2.ppf(1 - (1 - gamma) / 2, df=n-1)
conf_interval_variance = ((n - 1) * std_X**2 / chi2_upper, (n - 1) * std_X**2 / chi2_lower)

print("Интервальная оценка для дисперсии X:", conf_interval_variance)


2: График зависимости длины доверительного интервала от надежности при неизменном объеме выборки

In [ ]:
gamma_values = np.linspace(0.8, 0.99, 50)
interval_lengths_known_sigma = []
interval_lengths_unknown_sigma = []
interval_lengths_variance = []

for gamma in gamma_values:
    z = stats.norm.ppf(1 - (1 - gamma) / 2)
    t = stats.t.ppf(1 - (1 - gamma) / 2, df=n-1)
    chi2_lower = stats.chi2.ppf((1 - gamma) / 2, df=n-1)
    chi2_upper = stats.chi2.ppf(1 - (1 - gamma) / 2, df=n-1)
    
    interval_lengths_known_sigma.append(2 * z * sigma / np.sqrt(n))
    interval_lengths_unknown_sigma.append(2 * t * std_X / np.sqrt(n))
    interval_lengths_variance.append((n - 1) * std_X**2 * (1/chi2_lower - 1/chi2_upper))

plt.figure(figsize=(12, 6))
plt.plot(gamma_values, interval_lengths_known_sigma, label="Известная дисперсия")
plt.plot(gamma_values, interval_lengths_unknown_sigma, label="Неизвестная дисперсия")
plt.plot(gamma_values, interval_lengths_variance, label="Дисперсия")
plt.xlabel("Надежность (gamma)")
plt.ylabel("Длина доверительного интервала")
plt.legend()
plt.title("Зависимость длины доверительного интервала от надежности")
plt.show()

3: График зависимости длины доверительного интервала от объема выборки при неизменной надежности

In [ ]:
n_values = np.arange(5, 100, 5)
interv_lengths_known_sigma_n = []
interv_lengths_unknown_sigma_n = []
interv_lengths_variance_n = []

gamma = 0.99

for n in n_values:
    z = stats.norm.ppf(1 - (1 - gamma) / 2)
    t = stats.t.ppf(1 - (1 - gamma) / 2, df=n-1)
    chi2_lower = stats.chi2.ppf((1 - gamma) / 2, df=n-1)
    chi2_upper = stats.chi2.ppf(1 - (1 - gamma) / 2, df=n-1)
    
    interv_lengths_known_sigma_n.append(2 * z * sigma / np.sqrt(n))
    interv_lengths_unknown_sigma_n.append(2 * t * std_X / np.sqrt(n))
    interv_lengths_variance_n.append((n - 1) * std_X**2 * (1/chi2_lower - 1/chi2_upper))

plt.figure(figsize=(12, 6))
plt.plot(n_values, interv_lengths_known_sigma_n, label="Известная дисперсия")
plt.plot(n_values, interv_lengths_unknown_sigma_n, label="Неизвестная дисперсия")
plt.plot(n_values, interv_lengths_variance_n, label="Дисперсия")
plt.xlabel("Объем выборки (n)")
plt.ylabel("Длина доверительного интервала")
plt.legend()
plt.title("Зависимость длины доверительного интервала от объема выборки")
plt.show()

4: Моделирование M выборок и оценка надежности

In [ ]:
gamma_estimates = []

for _ in range(K):
    inside_count = 0
    for _ in range(M):
        X = np.random.normal(a, sigma, n)
        mean_X = np.mean(X)
        std_X = np.std(X, ddof=1)
        t = stats.t.ppf(1 - (1 - gamma) / 2, df=n-1)
        conf_interval = (mean_X - t * std_X / np.sqrt(n), mean_X + t * std_X / np.sqrt(n))
        if conf_interval[0] <= a <= conf_interval[1]:
            inside_count += 1
    gamma_estimates.append(inside_count / M)

gamma_hat = np.mean(gamma_estimates)
print("Точечная оценка надежности (gamma*):", gamma_hat)